In [ ]:
# Instalação das bibliotecas necessárias no Colab
!pip install transformers accelerate bitsandbytes sentence-transformers torch pandas

import torch
import time
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from typing import List, Dict

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 18.8 MB/s eta 0:00:00


In [ ]:
# Inserir token de acesso da conta Hugging Face.
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `BOFA` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cach

In [ ]:
# Defina o dispositivo (GPU, se disponível)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Executando no dispositivo: {device}")

# Configuração de Quantização
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Estes são os CHUNKS de texto que o retriever encontraria (SIMULADOS)
CHUNKS_RECUPERADOS: List[Dict] = [
    {
        "chunk": "O Conselho Monetário Nacional (CMN) dispõe que as sociedades de crédito, financiamento e investimento podem ser constituídas sob a forma de sociedade anônima (S.A.) ou limitada (Ltda.).",
        "source": "Resolução BCB nº 453",
        "chunk_index": 2, # Vizinho ANTERIOR ao chunk principal (3)
        "last_updated": "2025-07-24T10:00:00"
    },
    {
        "chunk": "As atividades principais incluem a concessão de crédito e a aquisição de direitos creditórios.",
        "source": "Resolução BCB nº 453",
        "chunk_index": 3, # O CHUNK PRINCIPAL encontrado pela similaridade
        "last_updated": "2025-07-24T10:00:00"
    },
    {
        "chunk": "A Política de Auditoria Interna do Banco Central do Brasil foi alterada pela Resolução BCB nº 453, com foco na transparência.",
        "source": "Resolução BCB nº 453",
        "chunk_index": 4, # Vizinho POSTERIOR ao chunk principal (3)
        "last_updated": "2025-01-30T09:00:00"
    },
    {
        "chunk": "O escopo de atuação do Coaf (Conselho de Controle de Atividades Financeiras) não se estende à fiscalização de cidadão ou de pessoa jurídica submetidos à esfera de atuação daquele conselho.",
        "source": "Resolução BCB nº 453",
        "chunk_index": 12, # Chunk não-vizinho, mas relevante para a outra parte da pergunta
        "last_updated": "2025-01-30T09:00:00"
    },
]

# 2. O Prompt (CRÍTICO para o RAG)
RAG_PROMPT_TEMPLATE = """
Você é um assistente de IA imparcial e analítico, especializado em regulamentações financeiras brasileiras.
Sua missão é responder à PERGUNTA DO USUÁRIO com base **EXCLUSIVAMENTE** no CONTEXTO fornecido.

INSTRUÇÕES DE RESPOSTA:

1.  **Acurácia e Síntese:** Revise todo o CONTEXTO. Sintetize as informações de forma clara e profissional.
2.  **Rastreabilidade (Citação Obrigatória):** Para cada parte da resposta, você **DEVE** incluir a citação exata (a [FONTE] original) que a suporta.
3.  **Restrição (Anti-Alucinação):** Se o CONTEXTO for insuficiente, não fizer sentido ou não contiver a informação pedida, você **DEVE** responder: "Desculpe, não encontrei informações relevantes nos documentos fornecidos para responder a essa pergunta específica."
4.  **Formato:** Mantenha a resposta direta ao ponto, com as fontes citadas entre colchetes.

--- CONTEXTO FORNECIDO ---
{contexto}
----------------------------

PERGUNTA DO USUÁRIO: {pergunta}

RESPOSTA CONCISA:
"""

Executando no dispositivo: cuda


In [ ]:
# Função de Pré-processamento e Contexto Aumentado
def formatar_contexto_rag(chunks: list[dict]) -> str:
    """
    Formata a lista de chunks em uma única string, incluindo metadados
    para que o LLM tenha a fonte e o timestamp.
    """
    context_str = ""
    for chunk in chunks:
        # Formato limpo e estruturado que o LLM pode entender
        context_str += f"""
[FONTE: {chunk['source']} | CHUNK ID: {chunk['chunk_index']} | ATUALIZAÇÃO: {chunk['last_updated']}]
{chunk['chunk']}
---
"""
    return context_str.strip()

def run_llm_generation(model_name: str, contexto: str, pergunta: str) -> str:
    """
    Função para carregar (uma vez), executar e descarregar um LLM
    com o prompt RAG formatado, garantindo a limpeza da VRAM.
    """
    print(f"[STATUS] Carregando {model_name}...")
    model = None
    tokenizer = None

    try:
        # Carregamento com quantização 4-bit para otimizar uso de VRAM no Colab
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            device_map=device,
            quantization_config=bnb_config
        )
        tokenizer = AutoTokenizer.from_pretrained(model_name)

        # Monta o prompt final
        formatted_prompt = RAG_PROMPT_TEMPLATE.format(
            contexto=contexto,
            pergunta=pergunta
        )

        # Cria o pipeline de geração
        pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=512, # Aumentado para garantir síntese complexa
            temperature=0.1,    # Baixo para respostas factuais e menos criativas
            do_sample=True,
        )

        # Gera a resposta
        output = pipe(formatted_prompt, return_full_text=False)
        response_text = output[0]['generated_text']

        # O LLM pode repetir parte do prompt, limpamos o texto gerado.
        cleaned_response = response_text.split("RESPOSTA CONCISA:")[-1].strip()

        return cleaned_response

    except Exception as e:
        return f"ERRO INTERNO DO LLM: Falha ao executar o modelo. Mensagem: {e}"

    finally:
        # Limpeza obrigatória de VRAM
        if model:
            del model
        if tokenizer:
            del tokenizer
        torch.cuda.empty_cache()
        print(f"[STATUS] {model_name} descarregado.")

In [ ]:
def executar_validacao_completa(model_name: str, chunks_recuperados: List[Dict]):

    # Preparação do contexto
    # Contexto Aumentado (usando todos os chunks para HIT/SÍNTESE)
    contexto_para_hit = formatar_contexto_rag(chunks_recuperados)

    # Contexto para o teste de ALUCINAÇÃO (Passamos um contexto real, mas perguntamos algo não contido)
    # A resposta de falha deve ser forçada.

    TEST_SUITE = [
        {"id": "P1", "pergunta": "Qual a forma jurídica e atividades das sociedades de crédito?", "contexto": contexto_para_hit, "tipo": "HIT_SIMPLES"},
        {"id": "P2", "pergunta": "Qual o prazo para o BCB emitir uma licença de funcionamento para uma fintech?", "contexto": contexto_para_hit, "tipo": "NÃO_SEI_DENTRO_DO_TÓPICO"},
        {"id": "P3", "pergunta": "Quem está fora do escopo de fiscalização do Coaf e qual o dado mais recente sobre isso?", "contexto": contexto_para_hit, "tipo": "SÍNTESE_E_METADADO"},
        {"id": "P4", "pergunta": "Quem ganhou a Copa do Mundo de 2022?", "contexto": contexto_para_hit, "tipo": "ALUCINAÇÃO_CONHECIMENTO_GERAL"},
    ]

    Tabela_Resultados = []

    print(f"\n=======================================================")
    print(f"       VALIDAÇÃO LLM: {model_name}        ")
    print(f"=======================================================")

    for test in TEST_SUITE:

        start_time = time.time()

        # Chamada ao LLM
        response = run_llm_generation(model_name, test['contexto'], test['pergunta'])

        elapsed_time = time.time() - start_time

        # Análise Automática

        # Validações dos critérios
        # O modelo de falha esperado é a resposta de Restrição do RAG_PROMPT_TEMPLATE
        expected_fail_phrase = "Desculpe, não encontrei informações relevantes"
        alucinacao_check = expected_fail_phrase in response
        citacao_check = "[" in response and "]" in response and not alucinacao_check

        # Conclusões para o relatório
        if alucinacao_check:
            alucinacao = "SIM (Resposta de falha correta)"
            acuracia_automatica = "N/A"
        elif citacao_check:
             alucinacao = "NÃO"
             acuracia_automatica = "AVALIAÇÃO HUMANA NECESSÁRIA (Fontes OK)"
        else:
             alucinacao = "NÃO"
             acuracia_automatica = "AVALIAÇÃO HUMANA NECESSÁRIA (Falhou em citar)"

        Tabela_Resultados.append({
            "CENÁRIO": test['id'] + " - " + test['tipo'],
            "PERGUNTA": test['pergunta'],
            "LATÊNCIA (s)": f"{elapsed_time:.2f}",
            "RESPOSTA DO LLM": response,
            "CITAÇÃO FORÇADA?": "SIM" if citacao_check else "NÃO",
            "ALUCINOU/FALHOU?": alucinacao,
            "ACURÁCIA FINAL": acuracia_automatica,
        })

        # Printa resultados intermediários para acompanhamento
        print(f"\n--- [CENÁRIO {test['id']} - {test['tipo']}] ---")
        print(f"LATÊNCIA: {elapsed_time:.2f} s")
        print(f"RESPOSTA: {response[:200]}...") # Printa apenas o começo da resposta

    # Exibe a tabela final para análise
    print("\n\n######################################################")
    print("## RELATÓRIO FINAL DO MODELO (Análise Humana Requerida) ##")
    print("######################################################")

    # Usa a biblioteca Pandas para formatar a tabela de forma legível no Colab
    try:
        import pandas as pd
        df = pd.DataFrame(Tabela_Resultados)
        # Define as colunas que o humano precisa analisar no Colab
        display_cols = ["CENÁRIO", "PERGUNTA", "LATÊNCIA (s)", "CITAÇÃO FORÇADA?", "ALUCINOU/FALHOU?", "ACURÁCIA FINAL"]
        pd.set_option('display.max_colwidth', 150)

        print(f"\nMODELO: {model_name}")
        display(df[display_cols])
        # O resultado completo (incluindo a resposta do LLM)
        print("\nRESPOSTAS COMPLETAS (Para cópia e análise):")
        for res in Tabela_Resultados:
             print(f"\n{res['CENÁRIO']}\n" + "-"*50 + f"\n{res['RESPOSTA DO LLM']}")

    except ImportError:
        print("\nInstale 'pandas' no Colab para melhor visualização: !pip install pandas")
        for res in Tabela_Resultados:
             print(f"\nCENÁRIO: {res['CENÁRIO']} | LATÊNCIA: {res['LATÊNCIA (s)']}\nResposta:\n{res['RESPOSTA DO LLM']}")

In [ ]:
# Lista de modelos
MODELOS_PARA_TESTE = ["mistralai/Mistral-7B-Instruct-v0.3", "microsoft/Phi-3-mini-4k-instruct", "meta-llama/Llama-3-8B-Instruct"]

for modelo in MODELOS_PARA_TESTE:
    executar_validacao_completa(modelo, CHUNKS_RECUPERADOS)


       VALIDAÇÃO LLM: mistralai/Mistral-7B-Instruct-v0.3        
[STATUS] Carregando mistralai/Mistral-7B-Instruct-v0.3...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

[STATUS] microsoft/Phi-3-mini-4k-instruct descarregado.

--- [CENÁRIO P4 - ALUCINAÇÃO_CONHECIMENTO_GERAL] ---
LATÊNCIA: 79.76 s
RESPOSTA: Desculpe, não encontrei informações relevantes nos documentos fornecidos para responder a essa pergunta específica.


### Solução 1:

Desculpe, não encontrei informações relevantes nos documentos forn...


######################################################
## RELATÓRIO FINAL DO MODELO (Análise Humana Requerida) ##
######################################################

MODELO: microsoft/Phi-3-mini-4k-instruct


,CENÁRIO,PERGUNTA,LATÊNCIA (s),CITAÇÃO FORÇADA?,ALUCINOU/FALHOU?,ACURÁCIA FINAL
0,P1 - HIT_SIMPLES,Qual a forma jurídica e atividades das sociedades de crédito?,385.14,NÃO,NÃO,AVALIAÇÃO HUMANA NECESSÁRIA (Falhou em citar)
1,P2 - NÃO_SEI_DENTRO_DO_TÓPICO,Qual o prazo para o BCB emitir uma licença de funcionamento para uma fintech?,2.43,NÃO,NÃO,AVALIAÇÃO HUMANA NECESSÁRIA (Falhou em citar)
2,P3 - SÍNTESE_E_METADADO,Quem está fora do escopo de fiscalização do Coaf e qual o dado mais recente sobre isso?,0.85,NÃO,NÃO,AVALIAÇÃO HUMANA NECESSÁRIA (Falhou em citar)
3,P4 - ALUCINAÇÃO_CONHECIMENTO_GERAL,Quem ganhou a Copa do Mundo de 2022?,79.76,NÃO,SIM (Resposta de falha correta),N/A



RESPOSTAS COMPLETAS (Para cópia e análise):

P1 - HIT_SIMPLES
--------------------------------------------------
ERRO INTERNO DO LLM: Falha ao executar o modelo. Mensagem: CUDA out of memory. Tried to allocate 96.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 62.12 MiB is free. Process 4186 has 14.68 GiB memory in use. Of the allocated memory 14.40 GiB is allocated by PyTorch, and 157.26 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

P2 - NÃO_SEI_DENTRO_DO_TÓPICO
--------------------------------------------------
ERRO INTERNO DO LLM: Falha ao executar o modelo. Mensagem: CUDA out of memory. Tried to allocate 96.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 60.12 MiB is free. Process 4186 has 14.68 GiB memory in use. Of the allo

,CENÁRIO,PERGUNTA,LATÊNCIA (s),CITAÇÃO FORÇADA?,ALUCINOU/FALHOU?,ACURÁCIA FINAL
0,P1 - HIT_SIMPLES,Qual a forma jurídica e atividades das sociedades de crédito?,0.35,NÃO,NÃO,AVALIAÇÃO HUMANA NECESSÁRIA (Falhou em citar)
1,P2 - NÃO_SEI_DENTRO_DO_TÓPICO,Qual o prazo para o BCB emitir uma licença de funcionamento para uma fintech?,0.24,NÃO,NÃO,AVALIAÇÃO HUMANA NECESSÁRIA (Falhou em citar)
2,P3 - SÍNTESE_E_METADADO,Quem está fora do escopo de fiscalização do Coaf e qual o dado mais recente sobre isso?,0.23,NÃO,NÃO,AVALIAÇÃO HUMANA NECESSÁRIA (Falhou em citar)
3,P4 - ALUCINAÇÃO_CONHECIMENTO_GERAL,Quem ganhou a Copa do Mundo de 2022?,0.24,NÃO,NÃO,AVALIAÇÃO HUMANA NECESSÁRIA (Falhou em citar)



RESPOSTAS COMPLETAS (Para cópia e análise):

P1 - HIT_SIMPLES
--------------------------------------------------
ERRO INTERNO DO LLM: Falha ao executar o modelo. Mensagem: meta-llama/Llama-3-8B-Instruct is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

P2 - NÃO_SEI_DENTRO_DO_TÓPICO
--------------------------------------------------
ERRO INTERNO DO LLM: Falha ao executar o modelo. Mensagem: meta-llama/Llama-3-8B-Instruct is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

P3 - SÍNTESE_E_METADADO
--------------------------------------------------
ERRO INTERNO D